<a href="https://colab.research.google.com/github/Gireesheruvaram/LLM-Beginer/blob/LLMadvkata6/LLMAdvkata6ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chromadb openai pandas fuzzywuzzy[speed]




In [ ]:

import openai
import chromadb
import pandas as pd
from fuzzywuzzy import process
import os


In [ ]:
import os
import openai
from google.colab import userdata
os.environ['OPENAI_API_KEY']=userdata.get('LLma')
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
# Initialize ChromaDB Persistent Client
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="movies")


In [ ]:
# Load Movie Dataset
data = pd.read_csv("Movies Dataset - Sheet1.csv")

In [ ]:
# Function to generate embeddings using OpenAI API
def get_openai_embedding(text):
    response = openai.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    # Access the embeddings correctly from the response object
    return response.data[0].embedding  # Corrected access method

# Insert Movies into ChromaDB
for movie in data.to_dict(orient='records'):
    embedding = get_openai_embedding(movie["Description"])
    collection.add(
        ids=[movie["Title"]],
        embeddings=[embedding],
        metadatas=[{
            "genre": movie["Genre"].lower(),
            "director": movie["Director"].lower(),
            "description": movie["Description"]
        }]
    )

print("Movies inserted successfully into ChromaDB!")


Movies inserted successfully into ChromaDB!


In [ ]:
# Extract Unique Genres & Directors
genre_list = data['Genre'].str.lower().unique().tolist()
director_list = data['Director'].str.lower().unique().tolist()

# Predefined User Query
user_query = "A thrilling action-packed movie with a deep storyline and suspenseful moments."

# Get embedding for the query
user_embedding = get_openai_embedding(user_query)


In [ ]:
# Apply Fuzzy Matching for Genre & Director
query_filter = {}

matched_genre, score = process.extractOne(user_query, genre_list) if genre_list else (None, 0)
if matched_genre and score > 70:
    query_filter["genre"] = matched_genre

matched_director, score = process.extractOne(user_query, director_list) if director_list else (None, 0)
if matched_director and score > 70:
    query_filter["director"] = matched_director

# Construct Filter for Query
where_filter = {}
if "genre" in query_filter and "director" in query_filter:
    where_filter = {"$and": [{"genre": query_filter["genre"]}, {"director": query_filter["director"]}]}
elif "genre" in query_filter:
    where_filter = {"genre": query_filter["genre"]}
elif "director" in query_filter:
    where_filter = {"director": query_filter["director"]}


In [ ]:
# Query ChromaDB
results = collection.query(
    query_embeddings=[user_embedding],
    n_results=5,
    where=where_filter if where_filter else None  # If where_filter is empty, pass None instead of {}
)

# Display Results
print("\nTop 5 Recommended Movies:")
if results["ids"]:
    for i, (title, metadata) in enumerate(zip(results["ids"][0], results["metadatas"][0])):
        print(f"{i+1}. {title} - {metadata['genre']} (Directed by {metadata['director']})")
        print(f"   Description: {metadata['description']}\n")
else:
    print("No results found. Try adjusting your query.")



Top 5 Recommended Movies:
1. 0 - science fiction (Directed by christopher nolan)
   Description: Inception is a 2010 science fiction action film written and directed by Christopher Nolan, who also produced the film with his wife, Emma Thomas. The film stars Leonardo DiCaprio as a professional thief who steals information by infiltrating the subconscious of his targets.

2. Inception - science fiction (Directed by christopher nolan)
   Description: Inception is a 2010 science fiction action film written and directed by Christopher Nolan, who also produced the film with his wife, Emma Thomas. The film stars Leonardo DiCaprio as a professional thief who steals information by infiltrating the subconscious of his targets.

3. The Matrix - science fiction (Directed by the wachowskis)
   Description: The Matrix is a 1999 science fiction action film written and directed by the Wachowskis. It depicts a dystopian future in which humanity is unknowingly trapped inside a simulated reality, the Ma